In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt 
import scipy

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test

import arrow # date format
import warnings; warnings.simplefilter('ignore') # do not disturb

In [ ]:
def calculate_sample_size_for_two_means(
    effect_size: float,
    sd: float,
    alpha: float = 0.05,
    power: float = 0.8,
    df: int = 1000,
) -> int:
    '''Given effect size, standard deviation, alpha (type I error), power (1 - type II error)
    and t-test df (degree of freedom), calculate minimum sample sizes required (in total),
    assuming equal-size cells, shared standard deviation and one-sided t-test.
    '''
    z1 = stats.t.ppf(q=1-alpha, df=df)
    z2 = stats.t.ppf(q=power, df=df)
    n = 2 * sd ** 2 * ((z1 + z2) / effect_size) ** 2
    
    return int(np.ceil(n) * 2)

In [ ]:
alpha = 0.05
power = 0.8

sample_size_df = pd.DataFrame()


#mean, std = 238809 , 58714
mean, std = df['roas'].mean(), df['roas'].std()

for i in range(0,len(desired_lift)):
    lift = desired_lift[i]
    # lift is applied to the original scale
    effect_size = (mean * lift)
    sample_size = calculate_sample_size_for_two_means(
        effect_size=effect_size,
        sd=std,
        alpha=alpha,
        power=power)
    sample_size_df.loc[i, "Lift"] = str(int(lift * 100)) + '%'
    sample_size_df.loc[i, "Effect Size"] = effect_size
    sample_size_df.loc[i, "Days"] = sample_size
    sample_size_df.loc[i, "Weeks"] = int(sample_size / 7)
    
sample_size_df

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import umap.umap_ as umap
import matplotlib.pyplot as plt

In [ ]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df.select_dtypes(include=['float64', 'int64']))

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(scaled_features)
df['cluster'] = kmeans.labels_
df.head()

In [ ]:
umap_reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
umap_embedding = umap_reducer.fit_transform(scaled_features)

In [ ]:
def find_sunday(date):
    monday = date - timedelta(days=date.weekday())
    # Subtract one day to get the Sunday
    sunday = monday - timedelta(days=1)
    return sunday

In [ ]:
import matplotlib.pylab as plt

from statsmodels.tsa.stattools import adfuller

import pmdarima as pm

from statsmodels.tsa.seasonal import seasonal_decompose

import statsmodels.api as sm

In [ ]:
model=sm.tsa.statespace.SARIMAX(
                                train_df['new_customer_count'], 
                                exog=train_df["spend_eur"],
                                order=(2, 1, 2),
                                seasonal_order=(2, 1, 2, 13)
                               )
results=model.fit()

test_df['forecast']= results.predict(
                                start="2023-12-24",
                                end="2024-03-10",
                                exog=test_df['spend_eur'],
                                dynamic=True
)

full_forecast = pd.concat([train_df, test_df])
full_forecast[['new_customer_count','forecast']].plot(figsize=(12,8))

In [ ]:
def PointEstimate(df1, df2, column_name):
    ## point estimate of the difference between two samples
    ## input should be two vectors
    df = df2[column_name] - df1[column_name]
    point_estimate = df.mean()
    std = df.std()
    return point_estimate, std

def StatSig(df1, df2, column_name): 
    pvalue = stats.ttest_ind(df1[column_name], df2[column_name]).pvalue
    alpha = 0.05
    if pvalue < alpha: 
        return True
    else: 
        return False
    
def ConfidenceInterval(df1, df2, column_name):
    # using 90% confidence interval 
    # point_estimate and lift are the same value
    point_estimate, std = PointEstimate(df1, df2, column_name)
    lift, lift_pctg = Lift(df1, df2, column_name)
    
    sample_size = len(df1.Day.unique())
    confidence = 0.90
    
    margin_of_error = std * stats.t.ppf((1 + confidence) / 2., sample_size -1 )
#     stats.t.interval(0.95, sample_size - 1, loc = point_estimate, scale = std)
    lower_bound = point_estimate - margin_of_error
    upper_bound = point_estimate + margin_of_error
    
    margin_of_error_pctg = margin_of_error / df1[column_name].mean()
    
    lower_bound_pctg = lift_pctg - margin_of_error_pctg
    upper_bound_pctg = lift_pctg + margin_of_error_pctg
    return (lower_bound, upper_bound), (lower_bound_pctg, upper_bound_pctg)

def Lift(df1, df2, column_name):
    # calculation the absolute and relative lift between the two arms
    mean1 = df1[column_name].mean()
    mean2 = df2[column_name].mean()
    lift = mean2 - mean1
#     lift_pctg = f"{lift / mean1:.0%}"
    lift_pctg = lift / mean1
    return lift, lift_pctg

In [ ]:
import numpy as np
from bayes_ab.experiments import PoissonDataTest
from bayes_ab.experiments import NormalDataTest
from bayes_ab.experiments import BinaryDataTest

# from bayesian_testing.experiments import BinaryDataTest
# from bayesian_testing.experiments import PoissonDataTest
# from bayesian_testing.experiments import NormalDataTest

# https://pypi.org/project/bayes-ab/
# https://pypi.org/project/bayes-ab/

# https://www.pymc.io/projects/examples/en/latest/causal_inference/bayesian_ab_testing_introduction.html
# https://support.dynamicyield.com/hc/en-us/articles/360035983014-Understanding-A-B-Test-Results#:~:text=Uplift%20is%20the%20percent%20difference,%2C%20the%20uplift%20is%2025%25.

In [ ]:
bayesian_test = PoissonDataTest()

bayesian_test.add_variant_data("control-installations", df_control.Installations)
bayesian_test.add_variant_data("test-installations", df_test.Installations)

# evaluate test
results = bayesian_test.evaluate()
# results
# print(pd.DataFrame(results).to_markdown(tablefmt="grid"))
# print(pd.DataFrame(results).set_index('variant').T.to_markdown(tablefmt="grid"))

# data = bayesian_test.data

# generate plots
# bayesian_test.plot_distributions(control='control-installations', test='test-installations', fname='binary_distributions_example.png')

In [ ]:
# generating some random data
rng = np.random.default_rng(52)
# random 1x1500 array of 0/1 data with 5.2% probability for 1:
data_a = rng.binomial(n=1, p=0.052, size=1500)
# random 1x1200 array of 0/1 data with 6.7% probability for 1:
data_b = rng.binomial(n=1, p=0.067, size=1200)

# initialize a test:
test = BinaryDataTest()

# add variant using raw data (arrays of zeros and ones):
test.add_variant_data("A", data_a)
test.add_variant_data("B", data_b)
# priors can be specified like this (default for this test is a=b=1/2):
# test.add_variant_data("B", data_b, a_prior=1, b_prior=20)

# add variant using aggregated data (same as raw data with 950 zeros and 50 ones):
# test.add_variant_data_agg("C", totals=1000, positives=50)

# evaluate test:
results = test.evaluate()
results
print(pd.DataFrame(results).set_index('variant').T.to_markdown(tablefmt="grid"))

In [ ]:
# the interpretation of the frequentist 95% confidence interval by dichotomizing it in 
# statistically significant or non-statistically significant, 
# hampering a proper discussion on the values, the width (precision) and the practical implications of such interval.
#  Interpretation of the Bayesian 95% confidence interval (which is known as credible interval): 
# there is a 95% probability that the true (unknown) estimate would lie within the interval, given the evidence provided by the observed data.


# I have done some research on the Bayesian approach Meta uses and compared it with the Frequentist approach we and Google use.
# The high level conclusion is that the confidence score and lower bound reported by Meta is comparable with and equivalent to the confidence level and confidence interval lower bound we report assuming that the true lift follows a normal distribution.
# The only subtle difference is the ideology between these two approach which is:
# Bayesian: Give observed data, there is a 99% (confidence score) probability that the true value of the lift lies within the credible region (conversions_incremental_lower, conversions_incremental_upper, in the export cell A20 and A21)
# Frequentist: If this experiment is repeated many times, 95 out of 100 times the lift will be higher than the 90% confidence interval's lower bound.
# This subtlety reflects the fundamental disagreement between the two on the definition of probability.
# Bayesianism believes probability is about the degree of certainty about certain events, which is essentially related to the observer's own knowledget about an event.
# Frequesntism believes probability is the frequency of certain events.
# They only give different results when there are  non Gaussian variables involved (i.e. when a variable doesn't follow normal distribution).

# https://trainline.slack.com/archives/C07CK7RMQ95/p1723130821592069
